In [ ]:
!pip install gputil ultralytics opencv-python pandas

from IPython import display
display.clear_output()
import ultralytics
from ultralytics import YOLO
ultralytics.checks()

Ultralytics 8.3.111 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 41.1/112.6 GB disk)


In [ ]:
import numpy as np
from scipy.spatial import distance
import cv2
from sklearn.cluster import KMeans

from collections import namedtuple
from collections import defaultdict
import math
from math import sqrt
import random
from PIL import Image

import pandas as pd
import time
import os

import shutil
from google.colab import files

import webcolors
from google.colab import drive

import time
import torch
import signal
import sys
import hashlib
from threading import Lock

In [ ]:
# Mount Google Drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
class PlayerIdentifier:
    def __init__(self, teams_colors_info, home_squad, away_squad, basic_colors, number_detection_model_path):
        self.teams = teams_colors_info
        self.Home = home_squad
        self.Away = away_squad
        self.BASIC_COLORS = basic_colors

        self.Point = namedtuple('Point', ('coords', 'n', 'ct'))
        self.Cluster = namedtuple('Cluster', ('points', 'center', 'n'))

        self.model_number_detection = YOLO(number_detection_model_path).to('cuda')

    def get_points(self, player_image):
        w, h = player_image.size
        return [self.Point(color, 3, count) for count, color in player_image.getcolors(w * h)]

    def euclidean(self, p1, p2):
        return np.linalg.norm(np.array(p1.coords) - np.array(p2.coords))

    def calculate_center(self, points, n):
        'Метод расчета центра кластера'
        coords = np.average([p.coords for p in points], axis=0, weights=[p.ct for p in points])
        return self.Point(coords.tolist(), n, 1)

    def kmeans(self, points, k, min_diff):
        'Метод определения кластеров на изображении'
        clusters = [self.Cluster([p], p, p.n) for p in random.sample(points, k)]
        while True:
            plists = [[] for _ in range(k)]
            for p in points:
                idx = min(range(k), key=lambda i: self.euclidean(p, clusters[i].center))
                plists[idx].append(p)

            diff = 0
            for i in range(k):
                center = self.calculate_center(plists[i], clusters[i].n)
                diff = max(diff, self.euclidean(clusters[i].center, center))
                clusters[i] = self.Cluster(plists[i], center, clusters[i].n)

            if diff < min_diff:
                break
        return clusters

    def rtoh(self, rgb):
        return '#{:02x}{:02x}{:02x}'.format(*rgb)

    def colorz(self, frame, x1, x2, y1, y2, n=3):
        'Метод определения доминирующего цвета на изображении игрока'
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        img = Image.fromarray(frame_rgb).crop((x1, y1, x2, y2))
        w, h = img.size
        cropped_image = img.crop((int(w * 0.3), int(h * 0.3), int(w * 0.7), int(h * 0.7)))

        clusters = self.kmeans(self.get_points(cropped_image), n, 1)
        return [self.rtoh(map(int, c.center.coords)) for c in clusters]

    def closest_basic_color(self, requested_color):
        'Метод определения ближайшего базового цвета из представленных'
        return min(self.BASIC_COLORS, key=lambda c: sum((a - b) ** 2 for a, b in zip(self.BASIC_COLORS[c], requested_color)))

    def get_basic_color_name(self, hex_color):
        'Метод получения названий базовых цветов на основе полученных результатов доминирующих цветов'
        return self.closest_basic_color(webcolors.hex_to_rgb(hex_color))

    def get_team(self, color_names, colors_list):
        'Метод определения команды игрока'
        home, away = set(self.teams['Home']), set(self.teams['Away'])
        colors = set(color_names)
        sim_home = len(colors & home) / len(colors | home)
        sim_away = len(colors & away) / len(colors | away)

        if sim_home > sim_away or (sim_home == sim_away and colors_list.count(self.teams['Home'][0]) > 1):
            return 'Home'
        return 'Away'

    def extracting_bounding_boxes(self, results):
        'Метод извлечения bounding box для цифр, найденных на изображении хоккеиста'
        new_results = [
            {
                'x1': box[0], 'y1': box[1], 'x2': box[2], 'y2': box[3],
                'confidence': conf, 'detected_class': cls,
                'class': results[0].names[int(cls)]
            }
            for box, cls, conf in zip(results[0].boxes.xyxy.tolist(), results[0].boxes.cls.tolist(), results[0].boxes.conf.tolist())
        ]
        return sorted(new_results, key=lambda item: item['x1'])

    def get_player(self, results, team):
        'Метод получения фамилии и номера игрока'
        number = ''.join(str(res['class']) for res in results)
        number = int(number) if number else None
        return (self.Home if team == 'Home' else self.Away).get(number, 'None'), number

    def player_identification(self, frame, x1, y1, x2, y2):
        'Метод Идентификации игрока'
        dominant_colors = self.colorz(frame, x1, x2, y1, y2, n=min(len(self.teams['Home']), len(self.teams['Away'])))
        color_list = [self.get_basic_color_name(color) for color in dominant_colors]
        team = self.get_team(set(color_list), color_list)

        player_im = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
        results = self.model_number_detection(player_im.crop((x1, y1, x2, y2)), conf=0.7, max_det=2)
        detection_data = self.extracting_bounding_boxes(results)
        player, number = self.get_player(detection_data, team)

        return player, number, team

In [ ]:
class HockeyAnalyzer:
    def __init__(self, video_directory, detection_model_path, segmentation_model_path, 
                 teams_colors_info, home_squad, away_squad, basic_colors, number_detection_model_path, path_to_save, tracker):
        # 1. Инициализация Google Drive
        self.drive_path = path_to_save
        os.makedirs(self.drive_path, exist_ok=True)

        # 2. Настройка обработки прерываний
        signal.signal(signal.SIGTERM, self._handle_interrupt)

        # 3. Инициализация моделей
        print("Загрузка моделей...")
        torch.cuda.empty_cache()
        self.model_detection = YOLO(detection_model_path).to('cuda')
        self.model_segmentation = YOLO(segmentation_model_path).to('cuda')

        self.player_identifier = PlayerIdentifier(teams_colors_info, home_squad, away_squad, basic_colors, number_detection_model_path)

        # 4. Получение списка видеофайлов
        self.video_files = [os.path.join(video_directory, f) for f in os.listdir(video_directory) if f.lower().endswith(('.mp4', '.avi', '.mov', '.mkv'))]
        self.video_files = sorted(self.video_files)

        if not self.video_files:
            raise ValueError("В указанной директории не найдено видеофайлов")

        self.video_prefixes = {}  # Словарь для хранения префиксов видеофайлов

        # 5. Параметры оптимизации
        self.checkpoint_interval = 300  # Сохранение каждые 300 кадров
        self.last_checkpoint = time.time()
        self.buffer_size = 60
        self.current_buffer = 0
        self.write_lock = Lock()

        # 6. Структуры данных (общие для всех видео)
        self.players_data = defaultdict(lambda: {
            "time_on_ice": 0,
            "last_seen": None,
            "last_position": None,
            "total_distance": 0,
            "video_writer": None,
            "video_path": None,
            "player_name": [],
            "player_number": [],
            "player_team": [],
            "frames": []
        })

        # 7. Настройка папок
        self.videos_folder = os.path.join(self.drive_path, 'player_videos')
        os.makedirs(self.videos_folder, exist_ok=True)
        self.fourcc = cv2.VideoWriter_fourcc(*'mp4v')

        # 8. Калибровка (будет выполнена при обработке первого видео)
        self.calibration_done = False  # Флаг, что калибровка выполнена
        self.pixels_to_meters = None
        self.width = None
        self.height = None
        self.fps = None
        print("Инициализация завершена")

    def _get_video_prefix(self, video_path):
        """Генерация уникального префикса для видеофайла"""
        if video_path not in self.video_prefixes:
            # Создаем хеш из пути к видео для уникального префикса
            hash_obj = hashlib.md5(video_path.encode())
            self.video_prefixes[video_path] = f"{int(hash_obj.hexdigest()[:8], 16):08x}_"
        return self.video_prefixes[video_path]

    def _handle_interrupt(self, signum, frame):
        """Обработка прерывания сессии"""
        print("\nПолучен сигнал прерывания! Сохраняем данные...")
        try:
            self._flush_buffers()
            self._save_statistics()

            for stats in self.players_data.values():
                if stats["video_writer"] is not None:
                    stats["video_writer"].release()

            if hasattr(self, 'cap') and self.cap and self.cap.isOpened():
                self.cap.release()

            print("Данные успешно сохранены при прерывании")
        except Exception as e:
            print(f"Ошибка при обработке прерывания: {str(e)}")
        finally:
            sys.exit(1)

    def _calculate_pixel_to_meter_ratio(self, frame):
        """Вычисление соотношения пикселей к метрам (только один раз)"""
        if self.calibration_done:
            return

        print("Выполнение калибровки по первому кадру...")
        results = self.model_segmentation(frame)
        for result in results:
            if result.masks:
                blue_lines = [mask.xy[0].astype(int) for mask in result.masks]
                if len(blue_lines) >= 2:
                    x1, y1 = blue_lines[0].mean(axis=0)
                    x2, y2 = blue_lines[-1].mean(axis=0)
                    distance_pixels = math.sqrt((x2-x1)**2 + (y2-y1)**2)
                    self.pixels_to_meters = math.sqrt(17.2**2 + 30**2) / distance_pixels
                    self.calibration_done = True
                    print(f"Коэффициент перевода пикселей в метры установлен: {self.pixels_to_meters:.6f}")
                    return

        raise ValueError("Не удалось определить линии для расчета масштаба")

    def _init_player_video(self, player_id):
        """Инициализация видеофайла для игрока"""
        video_path = os.path.join(self.videos_folder, f"player_{player_id}.mp4")

        # Если видео уже существует, дописываем в него
        if os.path.exists(video_path):
            # Закрываем предыдущий writer, если он был
            if self.players_data[player_id]["video_writer"] is not None:
                self.players_data[player_id]["video_writer"].release()

            # Открываем файл для дописывания
            writer = cv2.VideoWriter(video_path, self.fourcc, self.fps, (self.width, self.height))
        else:
            writer = cv2.VideoWriter(video_path, self.fourcc, self.fps, (self.width, self.height))

        if not writer.isOpened():
            raise RuntimeError(f"Не удалось создать видео для игрока {player_id}")

        self.players_data[player_id]["video_writer"] = writer
        self.players_data[player_id]["video_path"] = video_path

    def _process_player(self, player_id, bbox, frame, current_time, video_path):
        """Обработка данных игрока с учетом префикса видео"""
        prefix = self._get_video_prefix(video_path)
        unique_player_id = f"{prefix}{player_id}"

        x1, y1, x2, y2 = bbox

        name, number, team = self.player_identifier.player_identification(frame, x1, y1, x2, y2)

        if name not in [None, 'None']:
            self.players_data[unique_player_id]["player_name"].append(name)
            self.players_data[unique_player_id]["player_number"].append(number)
        if team is not None:
            self.players_data[unique_player_id]["player_team"].append(team)

        current_pos = ((x1 + x2)/2, (y1 + y2)/2)
        last_seen = self.players_data[unique_player_id]["last_seen"]

        if last_seen is not None:
            time_diff = current_time - last_seen
            self.players_data[unique_player_id]["time_on_ice"] += time_diff

            last_pos = self.players_data[unique_player_id]["last_position"]
            if last_pos is not None:
                distance = math.sqrt((current_pos[0]-last_pos[0])**2 + (current_pos[1]-last_pos[1])**2)
                self.players_data[unique_player_id]["total_distance"] += distance * self.pixels_to_meters / 1.63

        self.players_data[unique_player_id]["last_seen"] = current_time
        self.players_data[unique_player_id]["last_position"] = current_pos

        if self.players_data[unique_player_id]["video_writer"] is None:
            self._init_player_video(unique_player_id)

        frame_copy = frame.copy()
        cv2.rectangle(frame_copy, (int(x1), int(y1)), (int(x2), int(y2)), (0,255,0), 2)

        with self.write_lock:
            self.players_data[unique_player_id]["frames"].append(frame_copy)
            self.current_buffer += 1

        if self.current_buffer >= self.buffer_size:
            self._flush_buffers()

    def _flush_buffers(self):
        """Запись кадров из буферов"""
        with self.write_lock:
            for player_id, stats in self.players_data.items():
                if stats["frames"]:
                    if stats["video_writer"] is None:
                        self._init_player_video(player_id)
                    try:
                        for frame in stats["frames"]:
                            stats["video_writer"].write(frame)
                        stats["frames"].clear()
                    except Exception as e:
                        print(f"Ошибка записи для игрока {player_id}: {str(e)}")
                        self._init_player_video(player_id)
            self.current_buffer = 0

    def _get_most_likely_value(self, values, threshold=0.6):
        """
        Возвращает значение, которое встречается в threshold (60%) случаев,
        исключая None. Если нет значения, удовлетворяющего порогу,
        возвращает None.
        """
        # Фильтруем None значения
        filtered_values = [v for v in values if v is not None]
        if not filtered_values:
            return None

        # Считаем частоту каждого значения
        value_counts = {}
        total = len(filtered_values)
        for v in filtered_values:
            value_counts[v] = value_counts.get(v, 0) + 1

        # Находим значение с максимальной частотой
        most_common_value = max(value_counts.items(), key=lambda x: x[1])[0]
        frequency = value_counts[most_common_value] / total

        # Проверяем, удовлетворяет ли порогу
        if frequency >= threshold:
            return most_common_value
        return None

    def _save_statistics(self):
        """Сохранение статистики в Excel"""
        stats_path = os.path.join(self.drive_path, "play_by_play_shifts.xlsx")
        stats_list = []

        for unique_player_id, stats in self.players_data.items():
            if stats["time_on_ice"] >= 1:
                # Разделяем уникальный ID на префикс видео и оригинальный ID
                parts = unique_player_id.split('_')
                video_prefix = parts[0]
                original_id = parts[1] if len(parts) > 1 else None

                stats_list.append({
                    "Unique Player ID": unique_player_id,
                    "Name": self._get_most_likely_value(stats["player_name"]),
                    "Number": self._get_most_likely_value(stats["player_number"]),
                    "Team": self._get_most_likely_value(stats["player_team"]),
                    "Time (s)": round(stats["time_on_ice"]),
                    "Distance (m)": round(stats["total_distance"]),
                })

        # Сохраняем в Excel, добавляя к существующим данным
        if os.path.exists(stats_path):
            existing_df = pd.read_excel(stats_path)
            new_df = pd.DataFrame(stats_list)
            combined_df = pd.concat([existing_df, new_df])

            # Удаляем дубликаты, оставляя последнюю запись
            combined_df = combined_df.drop_duplicates(subset=['Unique Player ID'], keep='last')
            combined_df.to_excel(stats_path, index=False)
        else:
            pd.DataFrame(stats_list).to_excel(stats_path, index=False)

        print(f"Статистика сохранена: {stats_path}")

    def _process_single_video(self, video_path):
        """Обработка одного видеофайла"""
        print(f"\nНачало обработки видео: {os.path.basename(video_path)}")
        self.cap = cv2.VideoCapture(video_path)
        if not self.cap.isOpened():
            print(f"Ошибка открытия видеофайла: {video_path}")
            return

        self.width = int(self.cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        self.height = int(self.cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        self.fps = int(self.cap.get(cv2.CAP_PROP_FPS))

        # Калибровка только по первому кадру первого видео
        if not self.calibration_done:
            ret, frame = self.cap.read()
            if not ret:
                print(f"Не удалось прочитать первый кадр видео: {video_path}")
                self.cap.release()
                return

            try:
                self._calculate_pixel_to_meter_ratio(frame)
            except Exception as e:
                print(f"Ошибка калибровки: {str(e)}")
                self.cap.release()
                return
        else:
            # Для остальных видео пропускаем первый кадр
            ret, _ = self.cap.read()
            if not ret:
                print(f"Не удалось прочитать первый кадр видео: {video_path}")
                self.cap.release()
                return

        frame_count = 0
        start_time = time.time()

        try:
            while self.cap.isOpened():
                ret, frame = self.cap.read()
                if not ret:
                    break

                frame_count += 1
                current_time = frame_count / self.fps
                results = self.model_detection.track(frame, conf=0.6, persist=True, tracker=tracker, max_det=16, verbose=False)

                for result in results:
                    for box in result.boxes:
                        if int(box.cls) == 0 and box.id is not None:
                            self._process_player(int(box.id), box.xyxy[0].tolist(), frame, current_time, video_path)

                if time.time() - self.last_checkpoint > self.checkpoint_interval:
                    self._save_statistics()
                    self._flush_buffers()
                    self.last_checkpoint = time.time()

        except Exception as e:
            print(f"Ошибка при обработке видео {video_path}: {str(e)}")
            raise

        finally:
            duration = time.time() - start_time
            print(f"\nОбработано кадров: {frame_count}")
            print(f"Время обработки: {duration:.2f} сек ({frame_count/duration:.2f} FPS)")

            self._flush_buffers()
            if self.cap.isOpened():
                self.cap.release()

    def process_all_videos(self):
        """Обработка всех видеофайлов в директории"""
        print(f"Начало обработки {len(self.video_files)} видеофайлов...")
        total_start = time.time()

        for video_path in self.video_files:
            try:
                self._process_single_video(video_path)
            except Exception as e:
                print(f"Пропуск видео {video_path} из-за ошибки: {str(e)}")
                continue

        # Финализация после обработки всех видео
        self._finalize_processing(total_start)

    def _finalize_processing(self, start_time):
        """Завершение обработки всех видео"""
        print("\nЗавершение обработки всех видео...")

        self._flush_buffers()
        self._save_statistics()

        # Закрываем все видеопотоки
        for player_id, stats in self.players_data.items():
            if stats["video_writer"] is not None:
                stats["video_writer"].release()

        total_duration = time.time() - start_time
        print(f"\nОбщее время обработки: {total_duration:.2f} секунд")

        # Вывод информации о сохраненных видео
        video_files = []
        for player_id, stats in self.players_data.items():
            if stats["video_path"] and os.path.exists(stats["video_path"]):
                size = os.path.getsize(stats["video_path"])/(1024*1024)
                video_files.append((player_id, stats["video_path"], size))

        print("\nСохраненные видеофайлы:")
        for player_id, path, size in video_files:
            print(f"- Игрок {player_id}: {path} ({size:.2f} MB)")

In [ ]:
class HockeyAnalyzer_only_excel:
    def __init__(self, video_directory, detection_model_path, segmentation_model_path, 
                 teams_colors_info, home_squad, away_squad, basic_colors, number_detection_model_path, path_to_save, tracker):
        # 1. Инициализация Google Drive
        self.drive_path = path_to_save
        os.makedirs(self.drive_path, exist_ok=True)

        # 2. Настройка обработки прерываний
        signal.signal(signal.SIGTERM, self._handle_interrupt)

        # 3. Инициализация моделей
        print("Загрузка моделей...")
        torch.cuda.empty_cache()
        self.model_detection = YOLO(detection_model_path).to('cuda')
        self.model_segmentation = YOLO(segmentation_model_path).to('cuda')

        self.player_identifier = PlayerIdentifier(teams_colors_info, home_squad, away_squad, basic_colors, number_detection_model_path)

        # 4. Получение списка видеофайлов
        self.video_files = [os.path.join(video_directory, f) for f in os.listdir(video_directory) if f.lower().endswith(('.mp4', '.avi', '.mov', '.mkv'))]
        self.video_files = sorted(self.video_files)

        if not self.video_files:
            raise ValueError("В указанной директории не найдено видеофайлов")

        self.video_prefixes = {}  # Словарь для хранения префиксов видеофайлов

        # 5. Параметры оптимизации
        self.checkpoint_interval = 300  # Сохранение каждые 300 кадров
        self.last_checkpoint = time.time()
        self.write_lock = Lock()

        # 6. Структуры данных (общие для всех видео)
        self.players_data = defaultdict(lambda: {
            "time_on_ice": 0,
            "last_seen": None,
            "last_position": None,
            "total_distance": 0,
            "player_name": [],
            "player_number": [],
            "player_team": []
        })

        # 7. Калибровка (будет выполнена при обработке первого видео)
        self.calibration_done = False
        self.pixels_to_meters = None
        self.width = None
        self.height = None
        self.fps = None
        print("Инициализация завершена")

    def _get_video_prefix(self, video_path):
        """Генерация уникального префикса для видеофайла"""
        if video_path not in self.video_prefixes:
            hash_obj = hashlib.md5(video_path.encode())
            self.video_prefixes[video_path] = f"{int(hash_obj.hexdigest()[:8], 16):08x}_"
        return self.video_prefixes[video_path]

    def _handle_interrupt(self, signum, frame):
        """Обработка прерывания сессии"""
        print("\nПолучен сигнал прерывания! Сохраняем данные...")
        try:
            self._save_statistics()
            if hasattr(self, 'cap') and self.cap and self.cap.isOpened():
                self.cap.release()
            print("Данные успешно сохранены при прерывании")
        except Exception as e:
            print(f"Ошибка при обработке прерывания: {str(e)}")
        finally:
            sys.exit(1)

    def _calculate_pixel_to_meter_ratio(self, frame):
        """Вычисление соотношения пикселей к метрам (только один раз)"""
        if self.calibration_done:
            return

        print("Выполнение калибровки по первому кадру...")
        results = self.model_segmentation(frame)
        for result in results:
            if result.masks:
                blue_lines = [mask.xy[0].astype(int) for mask in result.masks]
                if len(blue_lines) >= 2:
                    x1, y1 = blue_lines[0].mean(axis=0)
                    x2, y2 = blue_lines[-1].mean(axis=0)
                    distance_pixels = math.sqrt((x2-x1)**2 + (y2-y1)**2)
                    self.pixels_to_meters = math.sqrt(17.2**2 + 30**2) / distance_pixels
                    self.calibration_done = True
                    print(f"Коэффициент перевода пикселей в метры установлен: {self.pixels_to_meters:.6f}")
                    return

        raise ValueError("Не удалось определить линии для расчета масштаба")

    def _process_player(self, player_id, bbox, frame, current_time, video_path):
        """Обработка данных игрока с учетом префикса видео"""
        prefix = self._get_video_prefix(video_path)
        unique_player_id = f"{prefix}{player_id}"

        x1, y1, x2, y2 = bbox

        name, number, team = self.player_identifier.player_identification(
            frame, x1, y1, x2, y2)

        if name not in [None, 'None']:
            self.players_data[unique_player_id]["player_name"].append(name)
            self.players_data[unique_player_id]["player_number"].append(number)
        if team is not None:
            self.players_data[unique_player_id]["player_team"].append(team)

        current_pos = ((x1 + x2)/2, (y1 + y2)/2)
        last_seen = self.players_data[unique_player_id]["last_seen"]

        if last_seen is not None:
            time_diff = current_time - last_seen
            self.players_data[unique_player_id]["time_on_ice"] += time_diff

            last_pos = self.players_data[unique_player_id]["last_position"]
            if last_pos is not None:
                distance = math.sqrt((current_pos[0]-last_pos[0])**2 + (current_pos[1]-last_pos[1])**2)
                self.players_data[unique_player_id]["total_distance"] += distance * self.pixels_to_meters / 1.63

        self.players_data[unique_player_id]["last_seen"] = current_time
        self.players_data[unique_player_id]["last_position"] = current_pos

    def _get_most_likely_value(self, values, threshold=0.6):
        """
        Возвращает значение, которое встречается в threshold (60%) случаев,
        исключая None. Если нет значения, удовлетворяющего порогу,
        возвращает None.
        """
        filtered_values = [v for v in values if v is not None]
        if not filtered_values:
            return None

        value_counts = {}
        total = len(filtered_values)
        for v in filtered_values:
            value_counts[v] = value_counts.get(v, 0) + 1

        most_common_value = max(value_counts.items(), key=lambda x: x[1])[0]
        frequency = value_counts[most_common_value] / total

        if frequency >= threshold:
            return most_common_value
        return None

    def _save_statistics(self):
        """Сохранение статистики в Excel"""
        stats_path = os.path.join(self.drive_path, "play_by_play_shifts.xlsx")
        stats_list = []

        for unique_player_id, stats in self.players_data.items():
            if stats["time_on_ice"] >= 1:
                parts = unique_player_id.split('_')
                video_prefix = parts[0]
                original_id = parts[1] if len(parts) > 1 else None

                stats_list.append({
                    "Unique Player ID": unique_player_id,
                    "Name": self._get_most_likely_value(stats["player_name"]),
                    "Number": self._get_most_likely_value(stats["player_number"]),
                    "Team": self._get_most_likely_value(stats["player_team"]),
                    "Time (s)": round(stats["time_on_ice"]),
                    "Distance (m)": round(stats["total_distance"])
                })

        if os.path.exists(stats_path):
            existing_df = pd.read_excel(stats_path)
            new_df = pd.DataFrame(stats_list)
            combined_df = pd.concat([existing_df, new_df])
            combined_df = combined_df.drop_duplicates(subset=['Player ID', 'Video Prefix'], keep='last')
            combined_df.to_excel(stats_path, index=False)
        else:
            pd.DataFrame(stats_list).to_excel(stats_path, index=False)

        print(f"Статистика сохранена: {stats_path}")

    def _process_single_video(self, video_path):
        """Обработка одного видеофайла"""
        print(f"\nНачало обработки видео: {os.path.basename(video_path)}")
        self.cap = cv2.VideoCapture(video_path)
        if not self.cap.isOpened():
            print(f"Ошибка открытия видеофайла: {video_path}")
            return

        self.width = int(self.cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        self.height = int(self.cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        self.fps = int(self.cap.get(cv2.CAP_PROP_FPS))

        if not self.calibration_done:
            ret, frame = self.cap.read()
            if not ret:
                print(f"Не удалось прочитать первый кадр видео: {video_path}")
                self.cap.release()
                return

            try:
                self._calculate_pixel_to_meter_ratio(frame)
            except Exception as e:
                print(f"Ошибка калибровки: {str(e)}")
                self.cap.release()
                return
        else:
            ret, _ = self.cap.read()
            if not ret:
                print(f"Не удалось прочитать первый кадр видео: {video_path}")
                self.cap.release()
                return

        frame_count = 0
        start_time = time.time()

        try:
            while self.cap.isOpened():
                ret, frame = self.cap.read()
                if not ret:
                    break

                frame_count += 1
                current_time = frame_count / self.fps

                results = self.model_detection.track(frame, conf=0.6, persist=True, tracker=tracker, max_det=16, verbose=False)

                for result in results:
                    for box in result.boxes:
                        if int(box.cls) == 0 and box.id is not None:
                            self._process_player(int(box.id), box.xyxy[0].tolist(), frame, current_time, video_path)

                if time.time() - self.last_checkpoint > self.checkpoint_interval:
                    self._save_statistics()
                    self.last_checkpoint = time.time()

        except Exception as e:
            print(f"Ошибка при обработке видео {video_path}: {str(e)}")
            raise

        finally:
            duration = time.time() - start_time
            print(f"\nОбработано кадров: {frame_count}")
            print(f"Время обработки: {duration:.2f} сек ({frame_count/duration:.2f} FPS)")
            if self.cap.isOpened():
                self.cap.release()

    def process_all_videos(self):
        """Обработка всех видеофайлов в директории"""
        print(f"Начало обработки {len(self.video_files)} видеофайлов...")
        total_start = time.time()

        for video_path in self.video_files:
            try:
                self._process_single_video(video_path)
            except Exception as e:
                print(f"Пропуск видео {video_path} из-за ошибки: {str(e)}")
                continue

        self._save_statistics()
        total_duration = time.time() - total_start
        print(f"\nОбщее время обработки: {total_duration:.2f} секунд")

In [ ]:
BASIC_COLORS = {
    'red': (255, 0, 0),
    'green': (0, 255, 0),
    'blue': (0, 0, 255),
    'yellow': (255, 255, 0),
    'cyan': (0, 255, 255),
    'magenta': (255, 0, 255),
    'black': (0, 0, 0),
    'white': (255, 255, 255),
    'gray': (128, 128, 128),
    'orange': (255, 165, 0),
    'purple': (128, 0, 128),
    'brown': (165, 42, 42)}

"\nBASIC_COLORS = {\n    'red': (255, 0, 0),\n    'green': (0, 255, 0),\n    'blue': (0, 0, 255),\n    'yellow': (255, 255, 0),\n    'cyan': (0, 255, 255),\n    'magenta': (255, 0, 255),\n    'black': (0, 0, 0),\n    'white': (255, 255, 255),\n    'gray': (128, 128, 128),\n    'orange': (255, 165, 0),\n    'purple': (128, 0, 128),\n    'brown': (165, 42, 42)}\n"

In [ ]:
# Класс HockeyAnalyzer
video_path = "/path/to/video"
detection_model_path = "/your_path/Player_Detection.pt"
segmentation_model_path = "/your_path/LinesDetection.pt"

#Класс PlayerIdentifier
number_detection_model_path = "/your_path/NumbersRec.pt"

path_to_save = "/path/to/save"
tracker = "/path/to/tracker.yaml"

teams_colors_info = {"Away": ["white", "red"], "Home": ["blue"]}

#Составы команд
Home = {1: "player_name"}
Away = {1: "player_name"}

analyzer = HockeyAnalyzer(video_path, detection_model_path, segmentation_model_path, teams_colors_info, Home, Away, 
                          BASIC_COLORS, number_detection_model_path, path_to_save, tracker)
analyzer.process_all_videos()

Выходные данные были обрезаны до нескольких последних строк (5000).
Speed: 0.6ms preprocess, 9.5ms inference, 0.8ms postprocess per image at shape (1, 3, 256, 128)

0: 256x160 (no detections), 11.0ms
Speed: 0.7ms preprocess, 11.0ms inference, 0.7ms postprocess per image at shape (1, 3, 256, 160)

0: 256x160 (no detections), 9.9ms
Speed: 0.7ms preprocess, 9.9ms inference, 0.7ms postprocess per image at shape (1, 3, 256, 160)

0: 256x160 1 6, 9.4ms
Speed: 0.7ms preprocess, 9.4ms inference, 1.7ms postprocess per image at shape (1, 3, 256, 160)

0: 256x128 (no detections), 11.7ms
Speed: 0.6ms preprocess, 11.7ms inference, 0.7ms postprocess per image at shape (1, 3, 256, 128)

0: 256x128 (no detections), 9.9ms
Speed: 0.6ms preprocess, 9.9ms inference, 0.6ms postprocess per image at shape (1, 3, 256, 128)

0: 256x160 (no detections), 10.2ms
Speed: 0.7ms preprocess, 10.2ms inference, 0.6ms postprocess per image at shape (1, 3, 256, 160)

0: 256x128 (no detections), 12.2ms
Speed: 0.5ms preproc